## Imports

In [ ]:
# pip install Flask pymongo transformers pandas numpy torch

In [ ]:
pip install matplotlib seaborn

In [ ]:
pip install pymongo[srv]

In [ ]:
# pip install requests

In [ ]:
pip install --upgrade transformers

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib
matplotlib.use('Agg')  # Use Agg backend, which is a non-interactive backend suitable for scripts.
import matplotlib.pyplot as plt


## Data

In [2]:
file_path = 'C:/Users/delis/Desktop/Jupyter Notebook Files/Projects/NoSQL/AC Sentiment/amazon-consumer.csv'
delimiter = ','  # Assuming the data is comma-separated

try:
    df = pd.read_csv(file_path, sep=delimiter, low_memory=False)
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")

In [3]:
data = df.copy()
df.head()

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    34660 non-null  object 
 1   name                  27900 non-null  object 
 2   asins                 34658 non-null  object 
 3   brand                 34660 non-null  object 
 4   categories            34660 non-null  object 
 5   keys                  34660 non-null  object 
 6   manufacturer          34660 non-null  object 
 7   reviews.date          34621 non-null  object 
 8   reviews.dateAdded     24039 non-null  object 
 9   reviews.dateSeen      34660 non-null  object 
 10  reviews.didPurchase   1 non-null      object 
 11  reviews.doRecommend   34066 non-null  object 
 12  reviews.id            1 non-null      float64
 13  reviews.numHelpful    34131 non-null  float64
 14  reviews.rating        34627 non-null  float64
 15  reviews.sourceURLs 

In [5]:
# Count the occurrences of each unique Clothing ID
product_reviews_count = df['id'].value_counts()

# Filter to find products that are reviewed multiple times
multiple_reviews = product_reviews_count[product_reviews_count > 1]

# Display the products that have been reviewed multiple times along with their review count
print(multiple_reviews)

id
AVphgVaX1cnluZ0-DR74    10966
AVpfl8cLLJeJML43AE3S     6619
AV1YE_muvKc47QAVgpwE     5056
AV1YnRtnglJLPUi8IJmV     3176
AVqkIhwDv8e3D1O-lebb     2814
AVqVGWLKnnc1JgDc3jF1     1685
AVpjEN4jLJeJML43rpUe     1038
AVpfpK8KLJeJML43BCuD      636
AVphPmHuilAPnD_x3E5h      580
AVsRjfwAU2_QcyX9PHqe      402
AV1YnR7wglJLPUi8IJmi      372
AVqVGWQDv8e3D1O-ldFr      270
AVpidLjVilAPnD_xEVpI      256
AVqVGZNvQMlgsOJE6eUY      212
AVqkIiKWnnc1JgDc3khH      158
AVpfIfGA1cnluZ0-emyp       73
AVqVGZO3nnc1JgDc3jGK       67
AVpftoij1cnluZ0-p5n2       51
AVqkIdntQMlgsOJE6fuB       30
AVpe7AsMilAPnD_xQ78G       19
AVpfBEWcilAPnD_xTGb7       19
AVzRlorb-jtxr-f3ygvQ       16
AVpf_4sUilAPnD_xlwYV       15
AVqkIj9snnc1JgDc3khU       14
AVpfiBlyLJeJML43-4Tp       13
AVphLY7v1cnluZ0-_Ty0       13
AVpf_znpilAPnD_xlvAF       10
AVpgdkC8ilAPnD_xsvyi       10
AVpggqsrLJeJML43O5zp        8
AVzvXXwEvKc47QAVfRhx        8
AVzoGHhAglJLPUi8GfzY        7
AVqkIhxunnc1JgDc3kg_        7
AVpg3q4RLJeJML43TxA_        6
AVqVGZN

Multiple products are reviewed multiple times

## Unique Review IDs

In [6]:
# Create a new column 'new_review_id' and initialize it with the existing review ID values
df['new_review_id'] = df['reviews.id']

# Find the rows with NaN review IDs
nan_rows = df['new_review_id'].isna()

# Generate unique IDs for those NaN rows using a range and add an appropriate prefix
df.loc[nan_rows, 'new_review_id'] = ['r' + str(i) for i in range(nan_rows.sum())]

# If you want to replace the original 'review_id' column, you can do:
df['reviews.id'] = df['new_review_id']
df.drop(columns=['new_review_id'], inplace=True)

In [7]:
# Group by product ID and count the number of reviews
reviews_count = df.groupby('id')['reviews.id'].count().reset_index()

# Rename the columns to prepare for merging
reviews_count.columns = ['id', 'numReviews']

# Merge the original DataFrame with the reviews_count DataFrame to add the 'numReviews' column
df = df.merge(reviews_count, on='id', how='left')


## Preprocessing

Instead of simly dropping NaN valued rows, we will now replace review.text values where they are null, with "No Review.". So that later after doing our own sentiment analysis, we can replace sentiment scores for such texts with their given review.rating values.

In [8]:
def preprocess_text(texts):
    cleaned_texts = []
    for text in texts:
        if isinstance(text, str):  # Check if text is a string
            text = text.lower()  # Convert to lowercase
            text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
            cleaned_texts.append(text)
        else:
            cleaned_texts.append('No Review.')  # Handle non-string entries
    return cleaned_texts

## Sentiment Analysis

We will be using 200 random rows

In [9]:
n = 150
df = df.sample(n=n)
df.to_csv('sample-{n}.csv')

### Instantiate Model

In [10]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### Encode & Calculate Sentiment

In [11]:
def get_sentiment_scores(reviews):
    sentiments = []
    for review in reviews:
        inputs = tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            return_tensors="pt", # Returns PyTorch tensors
            truncation=True,
            padding='max_length',
            max_length=512 # You might need to adjust the max_length based on your specific needs
        )

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            sentiment = torch.argmax(logits, dim=1).item()
            sentiments.append(sentiment)

    return sentiments

In [12]:
# Function to calculate the average sentiment score
def calculate_average_sentiment(df):
    # Group by product ID and calculate the mean of the 'Sentiment Score' for each group
    average_sentiments = df.groupby('id')['Sentiment_Score'].mean()

    # Map the average sentiments to a new column in the original DataFrame
    df['Average Sentiment Score'] = df['id'].map(average_sentiments)

    return df

### Load & Score Reviews

In [14]:
clean_reviews = preprocess_text(df['reviews.text'])

In [15]:
# Get sentiment scores for the reviews
sentiment_scores = get_sentiment_scores(clean_reviews)
df['Sentiment_Score'] = sentiment_scores

In [ ]:
def plot_sentiment_distribution(sentiment_scores):
    """
    Plots a histogram of the sentiment scores.
    """
    plt.figure(figsize=(10, 6))
    sns.countplot(sentiment_scores)
    plt.title('Distribution of Sentiment Scores')
    plt.xlabel('Sentiment Score')
    plt.ylabel('Number of Reviews')
    plt.xticks(ticks=range(len(set(sentiment_scores))), labels=[f"Sentiment {i}" for i in set(sentiment_scores)])
    plt.grid(axis='y')
    # plt.show()
    plt.savefig(sentiment_distribution)
    
# Plot the sentiment distribution for our dataset
plot_sentiment_distribution(sentiment_scores)

For the reviews which had no review.text values, we will now replace their calculated sentiment scores with their originally given review.rating scores.

In [ ]:
mask = df['reviews.text'] == "No Review."
df.loc[mask, 'Sentiment_Score'] = df.loc[mask, 'reviews.rating']

In [ ]:
df.to_csv('scored-{n}.csv')

We will then go onto calculating the average sentiment score based on the total reviews, for all products.

In [ ]:
df = calculate_average_sentiment(df)

And now we will replace the Null values for some of the review data with the string "None", and for the name column with "No Name".

In [ ]:
df['reviews.didPurchase'] = df['reviews.didPurchase'].fillna('None')
df['reviews.userCity'] = df['reviews.userCity'].fillna('None')
df['reviews.userProvince'] = df['reviews.userProvince'].fillna('None')

df['name'] = df['name'].fillna('No Name')

df.head()

## **CRUD Operations**

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

        uri = "mongodb+srv://delisamav3:<$Lindelani3>@cluster0.pd2aans.mongodb.net/?retryWrites=true&w=majority"

        # Create a new client and connect to the server
        client = MongoClient(uri, server_api=ServerApi('1'))

        # Send a ping to confirm a successful connection
        try:
            client.admin.command('ping')
            print("Pinged your deployment. You successfully connected to MongoDB!")
        except Exception as e:
            print(e)

In [ ]:
client = MongoClient('localhost', 27017)

In [ ]:
db = client['amazon_consumer_db']
products_collection = db['products']
reviews_collection = db['reviews']

Now we've connected to our database and its collections

####

Next, we want to drop duplicates based on the id column, but if there's a duplicate where one of the name values is "No Name", we want to prefer the other name. The following is will be our approach

Sort the DataFrame: First, sort the DataFrame so that rows with "No Name" for a particular id come after rows with actual names. This ensures that when duplicates are dropped based on id, the desired row (the one with an actual name) is kept.

Drop duplicates based on the 'id' column: After sorting, ww will then drop duplicates based on the id column. Since we've sorted in step 1, the first occurrence of a particular id will have the preferred name.


In [ ]:
# Rows with 'No Name' will be at the bottom for each id, so they will be dropped in the next step.
df_sorted = df.sort_values(by=['id', 'name'], ascending=[True, False], key=lambda col: col != 'No Name')

# Drop duplicates based on the 'id' column
df_unique = df_sorted.drop_duplicates(subset='id')

df_unique.to_csv('unique-{n}.csv', index=False)
df_unique.head()

## Benefits of This Design:
Scalability: By separating reviews from product details, you can handle a large number of reviews without hitting any document size limits.
Flexibility: Allows for querying and updating reviews independently of products.
Efficiency: Can reduce duplication of data (e.g., product details in every review).

## How to Implement Queries:
To get a product with its reviews: You can perform a query on the products collection and then either use the embedded review references to look up detailed review information or perform a join with the reviews collection (using MongoDB's $lookup operator) based on the review IDs.

To get a review with its associated product: You can query the reviews collection by review ID and then look up the associated product by the product ID.

## Schema:

Products Collection:

    {
        "product": {
            "id": "AVqkIhwDv8e3D1O-lebb",
            "name": "All-New Fire HD 8 Tablet...",
            "asins": "B01AHB9CN2",
            "brand": "Amazon",
            "categories": ["Electronics", "Tablets"],
            "keys": "...",
            "manufacturer": "Amazon",
            "average_sentiment_score": 0.5
            "numReviews": 2
        },
        "reviews": [
          {
            "date": "2021-01-01",
            "id": "r000012",
            "sentiment_score": 0.7
          },
          {
            "date": "2021-02-01",
            "id": "r001342",
            "sentiment_score": 0.3
          }
        ]
    }

Reviews Collection:

    {
          "id": "r000012",
          "productId": "AVqkIhwDv8e3D1O-lebb", // Reference to the product
          "date": "2017-01-13T00:00:00.000Z",
          "dateAdded": "2017-07-03T23:33:15Z",
          "dateSeen": ["2017-06-07T09:04:00.000Z", "2017-04-30T00:45:00.000Z"],
          "didPurchase": null,
          "doRecommend": true,
          "numHelpful": 0,
          "rating": 5,
          "sourceURLs": ["http://...", "http://..."],
          "text": "This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease.",
          "title": "Kindle",
          "userCity": null,
          "userProvince": null,
          "username": "Adapter",
          "sentimentScore": 4.7
    }

reviews are both embedded within the product document (for fast reads) and stored in a separate collection (for more detailed querying).

# Create

In [ ]:
def make_review_data_row(row):
    columns_to_keep = ['reviews.id', 'reviews.dateAdded', 'reviews.rating', 'Sentiment_Score']
    reviews = row[columns_to_keep].to_dict()
    
    # Mapping old column names to new ones
    column_name_map = {
        'reviews.id': 'id',
        'reviews.dateAdded': 'dateAdded',
        'reviews.rating': 'rating',
        'Sentiment_Score': 'sentimentScore'
    }
    
    renamed_reviews = {column_name_map[key]: value for key, value in reviews.items()}
    
    return renamed_reviews

In [ ]:
def make_review_data_dict(review):
    # Filtering the reviews DataFrame based on the product ID
    review_data = {
        "id": review.get('id'),
        "dateAdded": review.get('dateAdded'),
        "rating": review.get('rating'),
        "sentimentScore": review.get('sentimentScore')
    }
    return review_data

In [ ]:
def make_product_data(product_data, reviews_data):
    product_item = {
        "product": {
            "id": product_data['id'],
            "name": product_data['name'],
            "asins": product_data['asins'],
            "brand": product_data['brand'],
            "categories": product_data['categories'],
            "keys": product_data['keys'],
            "manufacturer": product_data['manufacturer'],
            "average_sentiment_score": product_data['Average Sentiment Score'],
            "numReviews": product_data['numReviews']
        },
        "reviews": reviews_data
    }
    return product_item

In [ ]:
def make_review(row):
    review = {
        "id": row['reviews.id'],
        "productId": row['id'],
        "date": row['reviews.date'],
        "dateAdded": row['reviews.dateAdded'],
        "dateSeen": row['reviews.dateSeen'].split(','),
        "didPurchase": row['reviews.didPurchase'],
        "doRecommend": row['reviews.doRecommend'],
        "helpfulScore": row['reviews.numHelpful'],
        "rating": row['reviews.rating'],
        "sourceURLs": row['reviews.sourceURLs'].split(','),
        "text": row['reviews.text'],
        "title": row['reviews.title'],
        "userCity": row['reviews.userCity'],
        "userProvince": row['reviews.userProvince'],
        "username": row['reviews.username'],
        "sentimentScore": row['Sentiment_Score']
    }
    return review

The following function will enable us with adding single item documents to our Product collection.

In [ ]:
def insert_product(product_item):
    try:
        db['products'].insert_one(product_item)
        print("Product inserted successfully.")
    except Exception as e:
        print("Error inserting product:", str(e))


Now for adding to the Reviews collection

In [ ]:
def insert_review(review_item):
    try:
        db['reviews'].insert_one(review_item)
        print("Review inserted successfully.")
    except Exception as e:
        print("Error inserting review:", str(e))


Replacing null values for .didPurchase, .userCity, and .userProvince with string "None" or "Null" ??

In [ ]:
for index, row in df_unique.iterrows():
    review_data = make_review_data_row(row)
    product = make_product_data(row, [review_data])
    review = make_review(row)
    
    insert_product(product)
    insert_review(review)

# Read

In [ ]:
def print_review(review_data):
    r_id = review_data['id']
    p_id = review_data['productId']
    date_add = review_data['dateAdded']
    score = review_data['sentimentScore']
    text = review_data['text']

    print(f"ID: {r_id}")
    print(f"Product ID: {p_id}")
    print(f"Date Added: {date_add}")
    print(f"Sentiment Score: {score}")
    print(f"Text: {text}")

In [ ]:
def get_review_ids(product_id):
    product = products_collection.find_one({'product.id': product_id})
    review_ids = [review['id'] for review in product['reviews']] if product else []

    return review_ids

In [ ]:
def get_review_data(review_id):
    review_data = reviews_collection.find_one({'id': review_id})

    return review_data

In [ ]:
def get_product_data(product_id):
    product_data = products_collection.find_one({'product.id': product_id})

    return product_data

In [ ]:
def get_all_product_ids():
    """
    Retrieves all the available product IDs from the database.
    """
    return db['products'].distinct('product.id')

In [ ]:
def get_all_review_ids():
    """
    Retrieves all the available review IDs from the database.
    """
    return db['reviews'].distinct('id')

We can print all existing product ids and then pick one to use for the demonstration

In [ ]:
get_all_product_ids()

In [ ]:
product_id = "AV1YnR7wglJLPUi8IJmi"

In [ ]:
product_data = get_product_data(product_id)

if product_data:
    product_name = product_data['product']['name']
    product_brand = product_data['product']['brand']
    first_review_date = product_data['reviews'][0]['dateAdded'] if product_data['reviews'] else None
    first_review_sentiment_score = product_data['reviews'][0]['sentimentScore'] if product_data['reviews'] else None

    print(f"Product Name: {product_name}")
    print(f"Product Brand: {product_brand}")
    print(f"First Review Date: {first_review_date}")
    print(f"First Review Sentiment Score: {first_review_sentiment_score}")
else:
    print("Product not found")

print("")

review_ids = get_review_ids(product_id)

if review_ids:
    print("Review IDs: ")
    print(review_ids)
    
else:
    print("Review IDs not found")

In [ ]:
review_data = get_review_data(review_ids[0])

if review_data:
    print_review(review_data)
else:
    print("Review data not found")

# Update

In [ ]:
def update_avgSentiment_after_insertion(product_id, sentiment_score):
    product = db['Products'].find_one({"product.id": product_id})
    if product:
        total_reviews = product["product"]["numReviews"]
        old_average = product["product"]["average_sentiment_score"]
        new_average = (old_average * total_reviews + sentiment_score) / (total_reviews + 1)
        db['Products'].update_one({"product.id": product_id}, {"$set": {"product.average_sentiment_score": new_average}})

def increase_number_of_reviews(product_id):
    db['Products'].update_one({"product.id": product_id}, {"$inc": {"product.numReviews": 1}})

In [ ]:
def decrease_number_of_reviews(product_id):
    db['Products'].update_one({"product.id": product_id}, {"$inc": {"product.numReviews": -1}})

def update_average_sentiment_score_after_deletion(product_id, sentiment_score_to_remove):
    product = db['Products'].find_one({"product.id": product_id})
    if product:
        total_reviews = product["product"]["numReviews"]
        old_average = product["product"]["average_sentiment_score"]
        # If only one review, set average sentiment score to 0
        if total_reviews == 1:
            new_average = 0
        else:
            new_average = (old_average * total_reviews - sentiment_score_to_remove) / (total_reviews - 1)
        db['Products'].update_one({"product.id": product_id}, {"$set": {"product.average_sentiment_score": new_average}})


In [ ]:
def insert_new_review(product_id, review):
    # Check if the product exists
    product = db['products'].find_one({"product.id": product_id})
    if not product:
        print("Product not found.")
        return

    # Check if the review already exists based on review id
    existing_review = db['reviews'].find_one({"id": review['id']})
    if existing_review:
        print(f"Review with id {review['id']} already exists.")
        return

    # If the review doesn't exist, insert it and update the product document accordingly
    try:
        # Updating the average sentiment score for the product and the number of reviews
        update_avgSentiment_after_insertion(product_id, review['sentimentScore'])
        increase_number_of_reviews(product_id)
        
        # Insert the review into the Reviews collection
        review['productId'] = product_id
        reviews_collection.insert_one(review)
        # print(review)

        # Add the review to the embedded reviews list inside the product document
        review = make_review_data_dict(review)
        # print(review)
        products_collection.update_one(
            {"product.id": product_id},
            {
                "$push": {"reviews": review},
                "$inc": {"product.numReviews": 1}  # Increase the count of reviews
            }
        )

        print("Review inserted successfully.")
    except Exception as e:
        print(f"Error inserting review: {str(e)}")


In [ ]:
review_data['id'] = "r47000"
if '_id' in review_data:
    del review_data['_id']
    
insert_new_review(product_id, review_data)

In [ ]:
review_data['id'] = "r300999A"
if '_id' in review_data:
    del review_data['_id']
    
insert_new_review(product_id, review_data)

In [ ]:
review_ids = get_review_ids(product_id)

if review_ids:
    print(review_ids)
else:
    print("Review IDs not found")

Now we want to update some specific value with regards to our stored data. In this case we will implement the functionality of updating the dateAdded for some specific review. The relating products document will also have to be updated.

In [ ]:
def update_review_date_in_product(product_id, review_id, new_date):
    """
    Updates the 'dateAdded' field for a review embedded in a product document.
    """
    result = db['products'].update_one(
        {"product.id": product_id, "reviews.id": review_id},
        {"$set": {"reviews.$.dateAdded": new_date}}
    )
    
    return result.modified_count > 0

In [ ]:
def update_review_date_added(review_id, new_date=None):
    """
    Updates the 'dateAdded' field for a review with the given review_id.
    If new_date is not provided, it updates with the current datetime.
    """
    if not new_date:
        new_date = datetime.now().isoformat()

    # Update the main review collection
    result = db['reviews'].update_one(
        {"id": review_id},
        {"$set": {"dateAdded": new_date}}
    )

    if result.modified_count:
        # Get the associated product ID for the review
        review = db['reviews'].find_one({"id": review_id})
        if review and 'productId' in review:
            product_id = review['productId']
            # Update the embedded review date in the related product document
            if update_review_date_in_product(product_id, review_id, new_date):
                print(f"Review {review_id}'s dateAdded in related product updated to {new_date}.")
            else:
                print(f"Failed to update embedded review {review_id}'s dateAdded in product {product_id}.")
        print(f"Review {review_id}'s dateAdded updated to {new_date}.")
    else:
        print(f"Review {review_id} not found or dateAdded was already {new_date}.")


In [ ]:
review_id = "r47000"

In [ ]:
review_data = get_review_data(review_id)

if review_data:
    print_review(review_data)
else:
    print("Review data not found")

In [ ]:
new_date = "2023-08-04T00:00:00.000Z"  # if not provided, current datetime is used
update_review_date_added(review_id, new_date)

In [ ]:
review_data = get_review_data(review_id)

if review_data:
    print_review(review_data)
else:
    print("Review data not found")

# Delete

To delete a review, we will:

1. Look up the review by its unique ID.

2. If found, use the associated productId to find the referenced product.

3. Decrease the number of reviews for that product.

4. Update the average sentiment score for that product.

5. Remove the review from the "reviews" field in the product document.

6. Delete the review document from the "reviews" collection.

In [ ]:
def delete_review(review_id):
    # First, look up the review by its unique ID
    review = db['reviews'].find_one({"id": review_id})
    
    if not review:
        print(f"No review found with ID {review_id}")
        return

    # Use the associated productId to find the referenced product
    product_id = review['productId']
    product = db['products'].find_one({"product.id": product_id})
    if not product:
        print(f"No product found with ID {product_id} for the review with ID {review_id}")
        return

    # Decrease the number of reviews for that product
    decrease_number_of_reviews(product_id)
    
    # Update the average sentiment score for that product
    sentiment_score_to_remove = review['sentimentScore']
    update_average_sentiment_score_after_deletion(product_id, sentiment_score_to_remove)

    # Remove the review from the "reviews" field in the product document
    db['products'].update_one(
        {"product.id": product_id},
        {"$pull": {"reviews": {"id": review_id}}}
    )

    # Finally, delete the review document from the "reviews" collection
    db['reviews'].delete_one({"id": review_id})

    print(f"Review with ID {review_id} deleted successfully.")


In [ ]:
delete_review(review_id)
review_ids = get_review_ids(product_id)

if review_ids:
    print('')
    print("Remaining Reviews:")
    print(review_ids)
else:
    print("Review IDs not found")

To delete a product and its associated reviews:

1. Look up the product by its unique ID.
2. If found, retrieve the list of associated review IDs or other identifiers if available.
3. Delete all reviews associated with the product.
4. Delete the product document from the "products" collection.

In [ ]:
def delete_product(product_id):
    # First, look up the product by its unique ID
    product = db['products'].find_one({"product.id": product_id})
    
    if not product:
        print(f"No product found with ID {product_id}")
        return

    # Delete all reviews associated with this product
    associated_reviews = db['reviews'].delete_many({"productId": product_id})
    if associated_reviews.deleted_count > 0:
        print(f"{associated_reviews.deleted_count} associated reviews deleted.")

    # Finally, delete the product document from the "products" collection
    db['products'].delete_one({"product.id": product_id})

    print(f"Product with ID {product_id} deleted successfully.")


In [ ]:
product_ids = get_all_product_ids()
print("All Product IDs:", product_ids)

review_ids = get_all_review_ids()
print("All Review IDs:", review_ids)

In [ ]:
delete_product(product_id)

In [ ]:
product_ids = get_all_product_ids()
print("All Product IDs:", product_ids)

review_ids = get_all_review_ids()
print("All Review IDs:", review_ids)

## **Client & App**

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

In [ ]:
client.close()

## Here's a breakdown of how this structure works:

Product Document: Contains information about a product, including an array of references to reviews (with minimal information such as date and sentiment score). This could be placed in a products collection.

Review Document: Contains detailed information about a review, including a reference to the product ID it's associated with. This could be placed in a reviews collection.


## Conclusion:
This design makes sense for a system where you expect to have many reviews for each product and want to keep the system scalable and normalized. It aligns well with typical NoSQL design patterns and takes advantage of MongoDB's flexibility in handling complex data structures. Make sure to carefully plan the indexes (e.g., on product ID, review ID) to optimize query performance.